# SEME - Search Engine Manipulation Effect

In this day and age information flows at an unmeasurable speed. Due to the rapid evolution of the internet, our society and social standards have changed drastically. It is fair to say that our lives are now highly influenced by Big Tech Internet companies such as Amazon, Google, Facebook and other social media platforms. 

Many of us have become customed to the use of search engines to search for information on a daily basis. The average person / general citizen relies on these search engines for common questions and daily wonderments, along with scientific facts and political preferences to name a few. But how the ranking of our search results appear to us is crucial. There have been many studies and experiments done on the subject that show how Internet search rankings have a significant impact on consumer choices, mainly because users trust and choose higher-ranked results more than lower-ranked results. 

In this Notebook we will demonstratehow the 


The US Presidential Election in 2016 left many people perplexed. 

![image.png](attachment:image.png)

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


(abstract tekið frá þeim um SEME, getum umorðað og tekið eitthvað ur þessum texta, eða vitnað í hann bara https://www.pnas.org/doi/10.1073/pnas.1419828112)
* Internet search rankings have a significant impact on consumer choices, mainly because users trust and choose higher-ranked results more than lower-ranked results. Given the apparent power of search rankings, we asked whether they could be manipulated to alter the preferences of undecided voters in democratic elections. Here we report the results of five relevant double-blind, randomized controlled experiments, using a total of 4,556 undecided voters representing diverse demographic characteristics of the voting populations of the United States and India. The fifth experiment is especially notable in that it was conducted with eligible voters throughout India in the midst of India’s 2014 Lok Sabha elections just before the final votes were cast. The results of these experiments demonstrate that (i) biased search rankings can shift the voting preferences of undecided voters by 20% or more, (ii) the shift can be much higher in some demographic groups, and (iii) search ranking bias can be masked so that people show no awareness of the manipulation. We call this type of influence, which might be applicable to a variety of attitudes and beliefs, the search engine manipulation effect. Given that many elections are won by small margins, our results suggest that a search engine company has the power to influence the results of a substantial number of elections with impunity. The impact of such manipulations would be especially large in countries dominated by a single search engine company.

In [4]:
seme = pd.read_csv("bias_alerts_raw.csv", sep='\t', header=0, encoding='latin1')

# seme.shape


# Data Pre-processing
When reading through this large dataset we came across some buggy information and some timer errors. 

First we needed to create a new dataset using only the essential variables.


//FamiliarCandidate -> how familiar they are with the candidate (scale 1-10)


In [54]:
seme_clean = seme[["GroupNumber","CandidateOrder","CandidateOrder1","CandidateOrder2",
              "Age","Gender","SearchesPerWeek","EverSearched","LastElection",
              "PoliticalView","FamiliarCandidate1","FamiliarCandidate2",
              "PreTestLikelyVoteBipolar",
              "PreTestVoteWhichCandidate","PreTestVoteEitherCandidate",
              "PostTestLikelyVoteBipolar",
              "PostTestLikelyVoteCandidate1","PostTestLikelyVoteCandidate2","PostTestVoteWhichCandidate","PostTestVoteEitherCandidate",
              "Bothered","BotheredText",	"NumberOfClicks","AvgSecPerPage","TotalSearchTime","Country","Fluency","ExpName",
              "TimePage1","TimePage2","TimePage3","TimePage4","TimePage5",
              'Timer1','Timer2','Timer3','Timer4','Timer5','Timer6','Timer7','Timer8','Timer9','Timer10','Timer11','Timer12',
                  'Timer13','Timer14','Timer15','Timer16','Timer17','Timer18','Timer19','Timer20','Timer21','Timer22','Timer23',
                  'Timer24','Timer25','Timer26','Timer27','Timer28','Timer29','Timer30'
              ]]
seme_clean.head()



,GroupNumber,CandidateOrder,CandidateOrder1,CandidateOrder2,Age,Gender,SearchesPerWeek,EverSearched,LastElection,PoliticalView,...,Timer21,Timer22,Timer23,Timer24,Timer25,Timer26,Timer27,Timer28,Timer29,Timer30
0,3,2,Ed Miliband,David Cameron,29,Male,9.0,Yes,Yes,Liberal (left),...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0
1,1,1,David Cameron,Ed Miliband,53,Female,30.0,Yes,Yes,Conservative (right),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,Ed Miliband,David Cameron,25,Male,5.0,Yes,Yes,Liberal (left),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2,Ed Miliband,David Cameron,26,Female,2.0,Yes,Yes,Liberal (left),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,1,David Cameron,Ed Miliband,30,Male,10.0,Yes,Yes,Moderate (centre),...,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0


To prevent any corruption in the data we need to delete timer errors. 

To find them we check if TimePage[1:5] + Timer[1:30] == TotalSearchTime

If this condition is not met it means that the testing for this user is not valid and therefore not presentable for our data analysis. 

að prófunin á þessum einstakling hafi því miður ekki farið rétt fram og því ekki hægt að nota þessi gögn. 



In [6]:
corrupt = []
for i, row in seme_clean.iterrows():
    first = row['TimePage1']+ row["TimePage2"]+ row["TimePage3"]+ row["TimePage4"]+ row["TimePage5"]
    second = row['Timer1']+ row['Timer2']+ row['Timer3']+ row['Timer4']+ row['Timer5']+ row['Timer6']+ row['Timer7']+ row['Timer8']+ row['Timer9']+ row['Timer10'] + row['Timer11'] + row['Timer12'] +row['Timer13'] +row['Timer14']+row['Timer15']+row['Timer16']+row['Timer17']+row['Timer18']+row['Timer19']+row['Timer20'] +row['Timer21']+row['Timer22']+row['Timer23']+row['Timer24']+row['Timer25']+row['Timer26']+row['Timer27']+row['Timer28']+row['Timer29']+row['Timer30']
    if first + second != row['TotalSearchTime']:
        corrupt.append(i)

print('Corrupt data is in rows: ', corrupt)



Corrupt data is in rows:  [49, 114, 122, 242, 542, 588, 869, 990, 1127, 1132, 1151, 1271, 1301, 1471, 1570, 1960, 2290, 2423, 3748, 3770]


In [7]:
print('Corrupt data is in rows: ', corrupt)
print("Line 49 have an error")
print(seme_clean.iloc[49,34:70].sum())
print(seme_clean.iloc[49,30],"\n")

print("Line 114 have an error")
print(seme_clean.iloc[114,34:70].sum())
print(seme_clean.iloc[114,30],"\n")

print("Line 48 shouldn't have an error")
print(seme_clean.iloc[48,34:70].sum())
print(seme_clean.iloc[48,30],"\n")

Corrupt data is in rows:  [49, 114, 122, 242, 542, 588, 869, 990, 1127, 1132, 1151, 1271, 1301, 1471, 1570, 1960, 2290, 2423, 3748, 3770]
Line 49 have an error
194.0
156.0 

Line 114 have an error
107.0
59.0 

Line 48 shouldn't have an error
904.0
904.0 



Then we identify the missing data. 

We see that some values seem to be missing, oddly enough the number of indices is almost always 3. The variable "Bothered Text" is expected to be missing that many types since subjects would only have input text there if the former variable "Bothered" had the response "Yes".  




In [55]:
# to clean
print(seme_clean.isnull().sum())

arr = np.where(pd.isnull(seme_clean['Gender']))[0]
# a list of variables with sum => 3
var_3 = ["Gender","SearchesPerWeek","EverSearched","LastElection",
              "PoliticalView","FamiliarCandidate1","FamiliarCandidate2",
              "PreTestLikelyVoteBipolar",
              "PreTestVoteWhichCandidate","PreTestVoteEitherCandidate",
              "PostTestLikelyVoteBipolar",
              "PostTestLikelyVoteCandidate1","PostTestLikelyVoteCandidate2","PostTestVoteWhichCandidate","PostTestVoteEitherCandidate",
              "Bothered","BotheredText",	"NumberOfClicks","AvgSecPerPage","TotalSearchTime","Country","Fluency"]

GroupNumber        0
CandidateOrder     0
CandidateOrder1    0
CandidateOrder2    0
Age                0
                  ..
Timer26            3
Timer27            3
Timer28            3
Timer29            3
Timer30            3
Length: 63, dtype: int64


We must find the 3 subjects with missing data to remove them from the dataset.

In [9]:
for var in var_3:
  t = np.where(pd.isnull(seme_clean[var]))[0] 
  for i in arr:
    for j in t:
      if i == j:
        missing = list(arr)
print()
print('Missing data is in rows: ', missing)



Missing data is in rows:  [1151, 1301, 1471]


In [10]:
print("The data of row 1151 there is alot of NaN")
print(seme_clean.iloc[1151,0:20].to_string())
print("\nThe data from the row 1152 where nothing is missing or is NaN")
print(seme_clean.iloc[1152,0:20].to_string())


The data of row 1151 there is alot of NaN
GroupNumber                                 2
CandidateOrder                              2
CandidateOrder1                   Ed Miliband
CandidateOrder2                 David Cameron
Age                                        25
Gender                                    NaN
SearchesPerWeek                           NaN
EverSearched                              NaN
LastElection                              NaN
PoliticalView                             NaN
FamiliarCandidate1                        NaN
FamiliarCandidate2                        NaN
PreTestImpressionCandidate1               NaN
PreTestImpressionCandidate2               NaN
PreTestLikelyVoteBipolar                  NaN
PreTestLikelyVoteCandidate1               NaN
PreTestLikelyVoteCandidate2               NaN
PreTestVoteWhichCandidate                 NaN
PreTestVoteEitherCandidate                NaN
PostTestImpressionCandidate1              NaN

The data from the row 1152 where noth



We also want to get rid of subjects who rated their english fluency from 1 to 10 to be less then 10, since we think those cases might make our analysis less clear. 




In [11]:
clean_fluency = list(seme_clean[seme_clean['Fluency'] <= 5].index)

For our representation of the matter we want to only focus on the users living in the United States. We there fore want to drop every subject who is not living in the US.

In [12]:
not_us = list(seme_clean[seme_clean['Country'] != 'United States'].index)


Now we remove allt the subjects with any missing data, corrupt data or low English fluency and the people who are not registered in USA. 

Finally we remove the columns used for finding the timer errors. 


In [56]:
from pandas.core.reshape.concat import concat
# concat all lists with data to remove
to_remove = ((clean_fluency + missing) + corrupt) + not_us

# print(len(to_remove))

sc = seme_clean.drop(to_remove)
sc = sc.drop(["TimePage1","TimePage2","TimePage3","TimePage4","TimePage5",
              'Timer1','Timer2','Timer3','Timer4','Timer5','Timer6','Timer7','Timer8','Timer9','Timer10','Timer11','Timer12',
                  'Timer13','Timer14','Timer15','Timer16','Timer17','Timer18','Timer19','Timer20','Timer21','Timer22','Timer23',
                  'Timer24','Timer25','Timer26','Timer27','Timer28','Timer29','Timer30'], axis=1)
sc.shape

(3315, 28)

# Analysis

want to show how often people use search engines each week. 




In [ ]:
# Age group 18 - 29 years
age_1 = sc['Age'].loc[sc['Age'] <= 29]
age1 = sc[sc['Age'] <= 29]
spw1 = [i for i in age1['SearchesPerWeek']]
# Age group 30 - 40 years
age_2 = sc['Age'].loc[(sc['Age'] > 29) & (sc['Age'] <= 39)]
age2 = sc[(sc['Age'] > 29) & (sc['Age'] <= 39)]
spw2 = [i for i in age2['SearchesPerWeek']]
# Age group 40 - 50 years
age_3 = sc['Age'].loc[(sc['Age'] > 39) & (sc['Age'] <= 49)] 
age3 = sc[(sc['Age'] > 39) & (sc['Age'] <= 49)]
spw3 = [i for i in age3['SearchesPerWeek']]
# Age group 50 - 60 years
age_4 = sc['Age'].loc[(sc['Age']) > 49 & (sc['Age'] <= 59)]
age4 = sc[(sc['Age']) > 49 & (sc['Age'] <= 59)]
spw4 = [i for i in age4['SearchesPerWeek']]
# Age group 60 - 70 years
age_5 = sc['Age'].loc[(sc['Age'] > 59) & (sc['Age'] <= 69)]
age5 = sc[(sc['Age'] > 59) & (sc['Age'] <= 69)]
spw5 = [i for i in age5['SearchesPerWeek']]
# Age group over 70 years
age_6 = sc['Age'].loc[sc['Age'] > 69]
age6 = sc[sc['Age'] > 69]
spw6 = [i for i in age6['SearchesPerWeek']]

#klára svo  fann ekki alveg nogu gott graph til að gera þetta
age1df = pd.DataFrame({'Age1': age_1, 'Spw1': spw1})
age2df = pd.DataFrame({'Age2': age_2, 'Spw2': spw2})
age3df = pd.DataFrame({'Age3': age_3, 'Spw3': spw3})
age4df = pd.DataFrame({'Age4': age_4, 'Spw4': spw4})
age5df = pd.DataFrame({'Age5': age_5, 'Spw5': spw5})
age6df = pd.DataFrame({'Age6': age_6, 'Spw6': spw6})

# Means for Age groups
means = {'Age_1': age1df['Spw1'].mean(), 'Age_2': age2df['Spw2'].mean(), 'Age_3': age3df['Spw3'].mean(), 'Age_4': age4df['Spw4'].mean(), 'Age_5': age5df['Spw5'].mean(), 'Age_6': age6df['Spw6'].mean()}

# sc.groupby(['Age']).sum().plot(kind='kde', y='SearchesPerWeek', autopct='%1.0f%%')
# Split into Group Studies 

og = sc[sc['ExpName'] == 'Original']
b1 = sc[sc['ExpName'] == 'BiasAlert1']
b2 = sc[sc['ExpName'] == 'BiasAlert2']
# ax = sc.plot.bar(x='Age', y='SearchesPerWeek', rot=0)
ax = age1.plot.hist(column=['SearchesPerWeek'], by='Gender', figsize=(10,8))

In [57]:
def f(row):
    if row['Age'] <= 29:
        val = "Group 1"
    elif row['Age'] > 29 & row['Age'] <= 39:
        val = 'Group 2'
    elif row['Age'] > 39 & row['Age'] <= 49:
        val = 'Group 3'
    elif row['Age'] > 49 & row['Age'] <= 59:
        val = 'Group 4'
    elif row['Age'] > 59 & row['Age'] <= 69:
        val = 'Group 5'
    else:
        val = 'Group 6'
    return val


sc['AgeGroup'] = sc.apply(f, axis=1)


# check for correlation between SearchPerWeek and Age

print(sc.corr())

                              GroupNumber  CandidateOrder       Age  \
GroupNumber                      1.000000       -0.010108 -0.025759   
CandidateOrder                  -0.010108        1.000000  0.003926   
Age                             -0.025759        0.003926  1.000000   
SearchesPerWeek                  0.004526        0.035478 -0.008288   
FamiliarCandidate1               0.023131       -0.011806  0.087352   
FamiliarCandidate2               0.029777       -0.052837 -0.006864   
PreTestLikelyVoteBipolar        -0.024265       -0.056591  0.016395   
PreTestVoteEitherCandidate            NaN             NaN       NaN   
PostTestLikelyVoteBipolar       -0.036634       -0.097996 -0.014529   
PostTestLikelyVoteCandidate1          NaN             NaN       NaN   
PostTestLikelyVoteCandidate2          NaN             NaN       NaN   
PostTestVoteEitherCandidate           NaN             NaN       NaN   
NumberOfClicks                   0.024305        0.032980  0.129402   
AvgSec

- Negative numbers on the bipolar candidate preference scale `PreTestLikelyVoteBipolar` and `PostTestLikelyVoteBipolar` indicate a preference for Ed Milliband, positive numbers indicate a preference for David Cameron.
- `PreVote`, `PostVote` are coded as 1 if the subject voted for the favored candidate:
    + For Group 1, 1 if vote for Cameron, else 0
    + For Group 2, 1 if vote for Miliband, else 0
    + For Group 3, 1 if vote for Miliband, else 0

- `VoteShift` is `PostVote - PreVote`. VMP is calculated as `VoteShift / PreVote`, see functions `vmp` for calculating VMP. `vmp.data` will calculate VMP for all factor levels in a group, e.g. gender: `vmp.data(study4, "Gender")`


In [64]:
# Split into Group Studies 

og = sc[sc['ExpName'] == 'Original']
b1 = sc[sc['ExpName'] == 'BiasAlert1']
b2 = sc[sc['ExpName'] == 'BiasAlert2']

g1 = sc[sc['GroupNumber'] == 1]
# Lets look at data from Experiementing with Group 1.
# The Test for Users in Group 1 was done with a search ranking bias in favour of David Cameron, against Ed Milliband.
# Before the test we want to see how many of undecided voters (percentage) were likely to vote for each candidate. 

milliband_pref = [(i >= 0) for i in g1['PreTestLikelyVoteBipolar']].count(True) 
# cameron_pref = 

  
# We check for correlation between 'PreTestLikelyVoteBipolar' and 'PostTestLikelyVoteBipolar' to 
#positive numbers in = preference for David Cameron
# print(milliband_pref)

print(sum([i for i in seme_clean['TimePage1']]))


602
nan


# pre_likely_bi2 = [(i >= 0) for i in g2['PreTestLikelyVoteBipolar']]
# milliband_pref2 = pre_likely_bi2.count(False)/len(pre_likely_bi2) 
# cameron_pref2 = pre_likely_bi2.count(True)/len(pre_likely_bi2) 

# plt.subplot(222) 
# data2 = np.array([milliband_pref2,cameron_pref2])
# mycolors2 = ["#F85888","#A890F0"]
# plt.pie(data2, labels=mylabels, shadow=True, autopct='%1.1f%%',colors=mycolors)
# # plt.legend(title='Preferance', loc='center')
# plt.title('Group 2')
# # plt.show()   

# pre_likely_bi3 = [(i >= 0) for i in g3['PreTestLikelyVoteBipolar']]
# milliband_pref3 = pre_likely_bi3.count(False)/len(pre_likely_bi3) 
# cameron_pref3 = pre_likely_bi3.count(True)/len(pre_likely_bi3) 

# plt.subplot(223) 
# data3 = np.array([milliband_pref3,cameron_pref3])
# plt.pie(data3, labels=mylabels, shadow=True, autopct='%1.1f%%',colors=mycolors)
# # plt.legend(title='Preferance', loc='lower left')
# plt.title('Group 3')



post_likely_bi = [(i >= 0) for i in g1['PostTestLikelyVoteBipolar']]
milliband_preff = post_likely_bi.count(False)/len(post_likely_bi) 
cameron_preff = post_likely_bi.count(True)/len(post_likely_bi) 
plt.subplot(221) 
data11 = np.array([milliband_preff,cameron_preff])
plt.pie(data11, labels=mylabels, shadow=True, autopct='%1.1f%%', colors=mycolors)
# plt.legend(title='Preferance', loc='center')
plt.title('Cameron Biased')
  

post_likely_bi2 = [(i >= 0) for i in g2['PostTestLikelyVoteBipolar']]
milliband_preff2 = post_likely_bi2.count(False)/len(post_likely_bi2) 
cameron_preff2 = post_likely_bi2.count(True)/len(post_likely_bi2) 
plt.subplot(222) 
data22 = np.array([milliband_preff2,cameron_preff2])
plt.pie(data22, labels=mylabels, shadow=True, autopct='%1.1f%%', colors=mycolors)
# plt.legend(title='Preferance', loc='center')
plt.title('Miliband Biased')


post_likely_bi3 = [(i >= 0) for i in g3['PostTestLikelyVoteBipolar']]
milliband_preff3 = post_likely_bi3.count(False)/len(post_likely_bi3) 
cameron_preff3 = post_likely_bi3.count(True)/len(post_likely_bi3) 
plt.subplot(223) 
data33 = np.array([milliband_pref3,cameron_pref3])
plt.pie(data33, labels=mylabels, shadow=True, autopct='%1.1f%%',colors=mycolors)
# plt.legend(title='Preferance', loc='lower left')
plt.title('Neutral')
# plt.tight_layout(3)
plt.show()   



(1085, 34) (1096, 34) (1134, 34)
